```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Linear Coriolis term

The coriolis parameter is modeled as a linear function
$$f = \tilde{f} + \beta y$$
where $y=0$ currently represent the lower boundary of the first non-ghost-cell of the domain.


This Notebook are for testing/verifying the implementation of this linear Coriolis term.

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, DataOutput, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_linearCoriolis_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

## Parameters

In [ ]:
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.5
g = 9.81
r = 0.0

# Eddy viscocity parameter
A = 1

# Coriolis parameters: f + beta * y
f = 0.02
beta = 0.0002/dy

wind = Common.WindStressParams(type=99)

# Boundary condions
# 1: reflective wall, 
# 2: periodic, 
# 3: open (flow relaxation scheme), 
bcSettings = 3


waterHeight = 60

#Calculate radius from center for plotting
x_center = dx*nx*0.5
y_center = dy*ny*0.5
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

reload(Common)

## Forward Backward Linear

In [ ]:
#Clean up old simulator if any:
if 'fbl_sim' in globals():
    fbl_sim.cleanUp()


if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
    validDomain = [None, None, 0, 0]
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
    validDomain = [1, 1, 0, 0]
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;

eta0 = np.zeros(dataShape, dtype=np.float32);
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);

#Create bump in to lower left of domain for testing
addCentralBump(eta0, nx-2, ny-2, dx, dy, ghosts)


#Initialize simulator
reload(FBL)
fbl_sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              coriolis_beta=beta,\
              wind_stress=wind, \
              boundary_conditions=boundaryConditions, \
              write_netcdf=False)
   
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                                u0[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                                v0[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]])
T = 200
def animate(i):
    if (i>0):
        t = fbl_sim.step(300.0)
    else:
        t = 0.0
    eta1, u1, v1 = fbl_sim.download()

    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                 u1[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                 v1[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]]);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

## Centered in time, centered in space

In [ ]:
#Centered in time, centered in space

#Clean up old simulator if any:
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()

ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    

# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
addCentralBump(eta0, nx, ny, dx, dy, ghosts)

#Initialize simulator
reload(CTCS)
ctcs_sim = CTCS.CTCS(cl_ctx, \
                     h0, eta0, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, A, \
                     coriolis_beta=beta, \
                     wind_stress=wind, \
                     write_netcdf=False, \
                     boundary_conditions=boundaryConditions )



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] , 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                                v0[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])

T = 200
def animate(i):
    if (i>0):
        t = ctcs_sim.step(300.0)
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1.0
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                 brighten*v1[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) +  "\tMax h: " + str(np.max(eta1 + 60)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

## CDKLM 16

In [ ]:
#Coriolis well balanced reconstruction scheme
if 'cdklm_sim' in globals():
    cdklm_sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
        
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');


# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
addCentralBump(h0, nx, ny, dx, dy, validDomain)

# Move it away from centre of domain
#for i in range(nx+10, 50, -1):
#    for j in range(50,ny+20-50):
#        h0[j,i] = h0[j+25,i-15]

#Initialize simulator
reload(CDKLM16)
cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                            h0, u0, v0, Bi, \
                            nx, ny, dx, dy, dt*4, \
                            g, f, r, \
                            coriolis_beta=beta, \
                            wind_stress=wind, \
                            boundary_conditions=boundaryConditions, \
                            reportGeostrophicEquilibrium=False)


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

T = 200
def animate(i):
    if (i>0):
        t = cdklm_sim.step(300.0)
    else:
        t = 0.0
    h1, u1, v1 = cdklm_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1
    
    plotter.plot(brighten*(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

## Kurganov-Petrova 2007

In [ ]:
#Coriolis well balanced reconstruction scheme
if 'kp07_sim' in globals():
    kp07_sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
        
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
addCentralBump(h0, nx, ny, dx, dy, validDomain)

#Initialize simulator
reload(KP07)
kp07_sim = KP07.KP07(cl_ctx, \
                     h0, Bi, u0, v0, \
                     nx, ny, dx, dy, 4*dt, \
                     g, f, r, \
                     coriolis_beta=beta, \
                     wind_stress=wind, \
                     boundary_conditions=boundaryConditions)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

T = 200
def animate(i):
    if (i>0):
        t = kp07_sim.step(300.0)
    else:
        t = 0.0
    h1, u1, v1 = kp07_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 2
    
    plotter.plot(brighten*(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim